## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-16-20-51-00 +0000,wsj,Education Department Delays Wage Garnishment o...,https://www.wsj.com/personal-finance/education...
1,2026-01-16-20-48-00 +0000,wsj,President Trump will issue a plan next week fo...,https://www.wsj.com/politics/policy/trump-plan...
2,2026-01-16-20-46-00 +0000,wsj,Supreme Court to Hear Bayer’s Challenge to Rou...,https://www.wsj.com/business/supreme-court-to-...
3,2026-01-16-20-46-00 +0000,wsj,Opinion | Martin Luther King’s Game Theory,https://www.wsj.com/opinion/martin-luther-king...
4,2026-01-16-20-46-00 +0000,wsj,Oil Futures Post Fourth Straight Weekly Gain,https://www.wsj.com/finance/commodities-future...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2344/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,70
440,iran,24
44,ice,23
219,new,18
8,president,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
181,2026-01-16-12-48-22 +0000,nypost,Venezuelan opposition leader María Corina Mach...,https://nypost.com/2026/01/16/us-news/venezuel...,137
1,2026-01-16-20-48-00 +0000,wsj,President Trump will issue a plan next week fo...,https://www.wsj.com/politics/policy/trump-plan...,137
303,2026-01-15-23-31-25 +0000,latimes,Venezuela opposition leader Machado says she '...,https://www.latimes.com/politics/story/2026-01...,134
283,2026-01-16-01-32-41 +0000,bbc,Trump threatens to invoke Insurrection Act to ...,https://www.bbc.com/news/articles/cwy15596p98o...,133
314,2026-01-15-22-58-57 +0000,nypost,Trump invites Schumer to White House to talk N...,https://nypost.com/2026/01/15/us-news/trump-in...,133


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
181,137,2026-01-16-12-48-22 +0000,nypost,Venezuelan opposition leader María Corina Mach...,https://nypost.com/2026/01/16/us-news/venezuel...
1,67,2026-01-16-20-48-00 +0000,wsj,President Trump will issue a plan next week fo...,https://www.wsj.com/politics/policy/trump-plan...
283,63,2026-01-16-01-32-41 +0000,bbc,Trump threatens to invoke Insurrection Act to ...,https://www.bbc.com/news/articles/cwy15596p98o...
217,59,2026-01-16-10-43-00 +0000,wsj,"With Military Action on Hold, the U.S. Puts Ne...",https://www.wsj.com/world/middle-east/with-mil...
55,44,2026-01-16-19-10-00 +0000,wsj,President Trump signaled that he may keep Nati...,https://www.wsj.com/politics/policy/trump-to-f...
183,41,2026-01-16-12-40-13 +0000,cbc,Canada strikes tariff deal with China as globa...,https://www.cbc.ca/news/politics/livestory/can...
249,41,2026-01-16-07-39-32 +0000,cbc,Former South Korea leader handed 5-year jail t...,https://www.cbc.ca/news/world/yoon-suk-yeol-ja...
97,40,2026-01-16-17-40-16 +0000,nyt,Agent Who Shot Renee Good in Minneapolis Was T...,https://www.nytimes.com/2026/01/16/us/ice-agen...
268,36,2026-01-16-03-30-40 +0000,latimes,Trump administration's demands for California'...,https://www.latimes.com/california/story/2026-...
10,34,2026-01-16-20-34-00 +0000,wsj,What has Trump done as president so far? The W...,https://www.wsj.com/politics/policy/donald-tru...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
